open the file that consists of the train data

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv('train.csv', delimiter=",", header=None)
xtrain=train_data.iloc[:,0:17].astype(float)
y_train=train_data.iloc[:,-1]

####split X_train into train and CV

#X_train, X_cv, y_train, y_cv = train_test_split(X,y,train_size=0.70, shuffle=True)
#train size always complement data

cv_data = pd.read_csv('cv.csv', delimiter=",", header=None)
xcv=cv_data.iloc[:,0:17].astype(float)
y_cv=cv_data.iloc[:,-1]
#display(pd.DataFrame(X_train))

Scale and optimize my data

In [3]:
from sklearn.preprocessing import StandardScaler

#data scaling
scaler = StandardScaler()

#### train ###
X_train=scaler.fit_transform(xtrain)
#X= scaler.fit_transform(X)
#np.std(X_train, axis=0)
#np.mean(X_train, axis=0)

##### test  ####
X_cv=scaler.fit_transform(xcv)
#np.std(X_test, axis=0)
#np.mean(X_test, axis=0)

In [4]:
from sklearn.model_selection import KFold # import KFold
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn import metrics



Below we are looking for different C and gamma values that will yield the highest percentage of the trained model. from the given 8 set values, each C value will be paired with each gamma value, and the observed highest pair will be taken and used as the optimal set values.
Using the values, we'll get the highest score value for 

In [5]:

#store the different gamma values
gamma_list=[2**(-11),2**(-9),2**(-7),2**(-5),2**(-3),2**(-1) ,2**1,2**3]
c_list=[2**(-3),2**(-1),2**1,2**3,2**5,2**7,2**9,2**11]
get_score=list()
gamma_c=list()
for g in gamma_list:
    for c in c_list:
        clf=SVC(kernel="rbf",gamma=g,C=c)
        clf.fit(X_train,y_train)
        get_score.append(clf.score(X_cv,y_cv))
        gamma_c.append((g,c))

In [6]:
rep_data =pd.DataFrame([gamma_c,get_score],index=["gamma  ,   C ","  Score accuracy"])
t=pd.DataFrame.transpose(rep_data)
t

,"gamma , C",Score accuracy
0,"(0.00048828125, 0.125)",0.532014
1,"(0.00048828125, 0.5)",0.703143
2,"(0.00048828125, 2)",0.756694
3,"(0.00048828125, 8)",0.800931
4,"(0.00048828125, 32)",0.820722
...,...,...
59,"(8, 8)",0.616997
60,"(8, 32)",0.622817
61,"(8, 128)",0.622817
62,"(8, 512)",0.622817


In [7]:
#highest percentage

g,c=gamma_c[get_score.index(max(get_score))]
print("Max score is: ",max(get_score),"\nWith gamma: ",g, "and C value: ",c)


Max score is:  0.8428405122235157 
With gamma:  0.001953125 and C value:  2048


k-fold


In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
kf=KFold(n_splits=6)

In [9]:

accuracies=[]
for train_index, test_index in kf.split(X_train):
    
    #train_index,test_index
    data_train, data_test = X_train[train_index], X_train[test_index]
    target_train, target_test = y_train[train_index], y_train[test_index]
    k=SVC(gamma=g, C=c)
    k.fit(data_train,target_train)
    ypred = k.predict(data_test)
    accuracy=accuracy_score(target_test,ypred)
    
    accuracies.append(accuracy)
average_accuracy = np.mean(accuracies)

print("Average accuarcy",average_accuracy)

Average accuarcy 0.8237518956081832


In [10]:
from sklearn.model_selection import cross_val_score
cv_score=cross_val_score(SVC(gamma=g, C=c),X_train,y_train,cv=kf)
np.mean(cv_score)

0.8237518956081832

In [11]:
svclassifier = SVC(gamma=g, C=c)  
svclassifier.fit(X_train, y_train)
y_pred = svclassifier.predict(X_cv) 
y_pred[0:300]


array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0], d

In [12]:
%store svclassifier


Stored 'svclassifier' (SVC)


In [13]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_cv, y_pred))  
print(classification_report( y_cv,y_pred) )

[[400  47]
 [ 88 324]]
              precision    recall  f1-score   support

           0       0.82      0.89      0.86       447
           1       0.87      0.79      0.83       412

    accuracy                           0.84       859
   macro avg       0.85      0.84      0.84       859
weighted avg       0.85      0.84      0.84       859

